# Will be performing Web scraping in this project on banks

## process
* Extract the data from the web
* Transform that data 
* Load the data to sqlite3 database

In [2]:
# import the relevant packages
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import numpy as np
import sqlite3



### all url and files that will be needed through out the project including variables are all definde below

In [3]:
url = 'https://web.archive.org/web/20230908091635 /https://en.wikipedia.org/wiki/List_of_largest_banks'
# initial table attribute to be fetch from the web
table_attribute = ['Name','MC_USD_Billion']
# Final table attribute 
ftable_attribute =['Name','MC_USD_Billion','MC_GBP_Billion','MC_EUR_Billion','MC_INR_Billion']
# where to save the file as csv
target_file = './Largest_bank_data.csv'
# database name to be use 
db_name = 'Banks.db'
# table name where the data will be stored in db
table_name='Largest_banks'
# log file 
log_file = 'code_log.txt'



#### # rate file that contain rate of some currency to dollars


In [4]:
rate_file = pd.read_csv('./exchange_rate.csv')

#### # convert the exchange rate read above to to dictionary

In [5]:
exchange_rate = rate_file.set_index('Currency').to_dict()['Rate']

### #Create the logfile which will handle writing the process with time in a file 

In [6]:
def log_progress(message):
    timestamp_format = '%Y-%d-%D-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open(log_file, 'a') as f:
        f.write(timestamp + ", " + message + "\n" )


### This part is where we extract the date from the url we declare at the top level using requests & BeautifulSoup

#### steps in this functions are 
* we fetch the page from the url as text
* we fetch the html contents using Beautiful soup
* we define our dataframe
* then we fetch all tables in the html
* we notice that the table which we find want to fetch data from is at index 0 so we fetch all row in that index
* we now loop through the table row(tr) 
* we fetch all data(td) and indicate it as col 
* we check if col id not 0 we loop through table row (tr) to fetch data in each row 
* then we store them in diction 
* then we create a dataframe with that dictionary of data
* lastly we joing the new dataframe created with the df created in line 3 inside the function

In [36]:
def extract(url, table_attribute):
    page = requests.get(url).text
    html = BeautifulSoup(page, 'html.parser')
    df = pd.DataFrame(columns=table_attribute)
    tables = html.find_all('tbody')
    rows = tables[1].find_all('tr')
    for row in rows:
        col = row.find_all('td')
        if col:
            name = str(col[1].find_all('a')[1].text)
            mc_usd_billion_str = col[2].contents[0].strip().replace('\n', ',')
            mc_usd_billion_str = mc_usd_billion_str.replace(',', '')
            data_dic = {'Name': name,
                        'MC_USD_Billion': float(mc_usd_billion_str)}
            df1 = pd.DataFrame(data_dic, index=[0])
            df = pd.concat([df, df1], ignore_index=True)

    return df

#### #TRansform the data in billion

In [37]:
def transform(df, csv_path):
    df['MC_GBP_Billion'] = [
        np.round(x * exchange_rate['GBP'], 2) for x in df['MC_USD_Billion']]
    df['MC_INR_Billion'] = [
        np.round(x * exchange_rate['INR'], 2) for x in df['MC_USD_Billion']]
    df['MC_EUR_Billion'] = [
        np.round(x * exchange_rate['EUR'], 2) for x in df['MC_USD_Billion']]
    return df

#### Load the data into csv file

In [38]:
def load_to_csv(df, output_path):
    df.to_csv(output_path)

#### Load the data into db

In [39]:
def load_to_db(df, sql_connection, table_name):
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

#### create a function for running query 

In [40]:
def run_query(query_statement, sql_connection):
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

In [41]:
# Log the initialization of the ETL process 
log_progress("ETL Job Started") 
 
# Log the beginning of the Extraction process 
log_progress("Extract phase Started") 
extracted_data = extract(url,table_attribute) 
 
# Log the completion of the Extraction process 
log_progress("Extract phase Ended") 

In [42]:
# # Log the beginning of the Transformation process 
log_progress("Transform phase Started") 
transformed_data = transform(extracted_data,target_file) 
print("Transformed Data") 
print(transformed_data) 

# Log the completion of the Transformation process 
log_progress("Transform phase Ended") 

Transformed Data
                                               Name  MC_USD_Billion  \
0   Industrial and Commercial Bank of China Limited         5742.86   
1                           China Construction Bank         5016.81   
2                        Agricultural Bank of China         4919.03   
3                                     Bank of China         4192.12   
4                                    JPMorgan Chase         3868.24   
..                                              ...             ...   
95                          National Bank of Canada          312.67   
96                                  Macquarie Group          313.47   
97                      Nationwide Building Society          311.93   
98                                Raiffeisen Gruppe          303.50   
99                             First Abu Dhabi Bank          302.22   

    MC_GBP_Billion  MC_INR_Billion  MC_EUR_Billion  
0          4594.29       476370.24         5340.86  
1          4013.45      

In [43]:

# Log the beginning of the Loading process 
log_progress("Load phase Started") 
load_to_csv(transformed_data,target_file)

# Log the completion of the Loading process 
log_progress("Load phase Ended") 

In [44]:
conn = sqlite3.connect(db_name)

 
# Log the beginning of the Loading process 
log_progress("Load to db phase Started") 
load_to_db(transformed_data,conn,table_name) 
 
# Log the completion of the Loading process 
log_progress("Load to db phase Ended") 

In [45]:

run_query(f"SELECT * FROM Largest_banks",conn)
run_query(f"SELECT AVG(MC_GBP_Billion) FROM Largest_banks",conn)
run_query(f"SELECT Name from Largest_banks LIMIT 5",conn)
# Log the completion of the ETL pr

SELECT * FROM Largest_banks
                                               Name  MC_USD_Billion  \
0   Industrial and Commercial Bank of China Limited         5742.86   
1                           China Construction Bank         5016.81   
2                        Agricultural Bank of China         4919.03   
3                                     Bank of China         4192.12   
4                                    JPMorgan Chase         3868.24   
..                                              ...             ...   
95                          National Bank of Canada          312.67   
96                                  Macquarie Group          313.47   
97                      Nationwide Building Society          311.93   
98                                Raiffeisen Gruppe          303.50   
99                             First Abu Dhabi Bank          302.22   

    MC_GBP_Billion  MC_INR_Billion  MC_EUR_Billion  
0          4594.29       476370.24         5340.86  
1          40